This demo uses the implemented story RSPY-120.

ASSUMPTION: All the files from CADIP and ADGS mockup stations are already ingested and well published in the catalog. The ingestion of these files can be performed by running the demo prefect_flows_with_catalog_support from sprint_7 prior running this demo

## Configuration

In [ ]:
import getpass
import os

# S3 access
if not os.getenv("S3_ACCESSKEY"):
    os.environ["S3_ACCESSKEY"] = getpass.getpass(f"Enter S3 access key for {os.environ['S3_ENDPOINT']!r}:")
if not os.getenv("S3_SECRETKEY"):
    os.environ["S3_SECRETKEY"] = getpass.getpass(f"Enter S3 secret key for {os.environ['S3_ENDPOINT']!r}:")
    
# API key authentication (not on local mode)
if (os.getenv("RSPY_LOCAL_MODE") != "1") and (not os.getenv("RSPY_APIKEY")):
    os.environ["RSPY_APIKEY"] = getpass.getpass(f"Enter your API key from {os.environ['RSPY_WEBSITE']!r}:")


In [ ]:
# Set local or cluster configuration
import os

if os.getenv("RSPY_LOCAL_MODE") == "1":
    url_catalog = "http://rs-server-catalog:8000"
    url_dpr_sim = "http://dpr-simulator:8000"
    HEADERS={}
    local_mode = True
else:
    url_catalog = os.environ["RSPY_WEBSITE"]
    # the dpr mockup can be run inside the cluster only
    url_dpr_sim = "http://mockup-processor-dpr-svc.processing.svc.cluster.local:8080"
    HEADERS={"headers": {"x-api-key": os.environ["RSPY_APIKEY"]}}
    local_mode = False

print(f"Using url for catalog: {url_catalog}")
print(f"Using url for dpr-simulator: {url_dpr_sim}")

import requests
import json

Install the needed libraries

In [ ]:
!pip install boto3
if local_mode:
    try:
        import rs_workflows
    except ModuleNotFoundError:
        !(cd $RSPY_WHL_DIR && pip install rs_client_libraries-*.whl) # install rs-client if missing
        !opentelemetry-bootstrap -a install # install opentelemetry instrumentation for deps

In [ ]:
# Instrument these notebook cells with OpenTelemetry (optional)
from opentelemetry import trace
otel_tracer = trace.get_tracer("my.notebook.trace")

In [ ]:
# Check if there are the same products as this demo will process in the final catalog bucket

import boto3
import os

if local_mode:
    delete_files = True
else:
    delete_files = input("Before running the demo, check if previous results are in the final bucket. If these are to be found, should they be removed (optional)? (y/n): ").lower().strip() == 'y'

s3_dir = "PRODUCTS_DEMO_SPRINT_8"

s3_session = boto3.session.Session()
s3_client = s3_session.client(
    service_name="s3",
    aws_access_key_id=os.environ["S3_ACCESSKEY"],
    aws_secret_access_key=os.environ["S3_SECRETKEY"],
    endpoint_url=os.environ["S3_ENDPOINT"],
    region_name=os.environ["S3_REGION"],
)

bucket_name = "rs-cluster-catalog"
none_found = True
for bucket in s3_client.list_buckets()["Buckets"]:
    if bucket_name != bucket["Name"]:
        continue
    if 'Contents' in s3_client.list_objects(Bucket=bucket_name):
        objects = s3_client.list_objects(Bucket=bucket_name)['Contents']
        for obj in objects:
            if s3_dir in obj["Key"]:                
                if delete_files:
                    print(f"Deleting {obj['Key']}")
                    s3_client.delete_object(Bucket=bucket_name, Key=obj['Key'])
                else:                          
                    print(f"{obj['Key']} modified on {obj['LastModified']}") 
                    none_found = False
print("No products found in the catalog bucket") if none_found else print("\nProducts found in the catalog bucket")

In [ ]:
# Clean previous executions
requests.delete(f"{url_catalog}/catalog/collections/DemoUser:s1_dpr", **HEADERS)

In [ ]:
# Assumption: all the files from CADIP and ADGS server are well ingested in the catalog  as well as in the buckets
# Note: the S3_ACCESSKEY, S3_SECRETKEY and S3_ENDPOINT are given in the docker-compose.yml file.
with otel_tracer.start_as_current_span("nb.run-flow-s1_l0") as span:
    from rs_workflows.s1_l0 import (
        s1_l0_flow,
        PrefectS1L0FlowConfig,
    )
    user = "DemoUser"
    mission = "s1"
    session_id = "S1A_20200105072204051312"
    s3_storage = f"s3://rs-cluster-final/{s3_dir}/"
    temp_s3_storage = f"s3://rs-cluster-temp/{s3_dir}/"
    # the type of the prducts that the DPR should process
    product_types = ["S1SEWRAW", "S1SIWRAW", "S1SSMRAW", "S1SWVRAW"]
    s1_l0_flow(
            PrefectS1L0FlowConfig(
                user,
                url_catalog,
                url_dpr_sim,
                mission,
                session_id,
                product_types,
                s3_storage,
                temp_s3_storage,
                os.environ.get("RSPY_APIKEY", None),
            ),
        )